## risk analysis and mitigation
1. channel tuple frequency analysis
2. channel value assignment
3. vector value assignment
4. similarity calculation
    - risk score

In [2]:
import numpy as np
import json 

In [3]:
# channel tuple frequency analysis
file = open("../1_application_analysis/1_Intra_app_Analysis/trigger-action.JSON","r") # read all the capability extrapolated from the apps
trigger_action = json.load(file)
print("tot app analysed:",len(trigger_action.keys()))

tot app analysed: 86


In [4]:
#--- attention! now don't remove the duplicates pair trigger-action
list_trigger_action = []
for app in trigger_action.keys():         # for every app analysed
    for trigger in trigger_action[app].keys():      # for every trigger channel
            for action in trigger_action[app][trigger]:
                list_trigger_action.append([trigger, action]) #save capability (the channel) and action
list_trigger_action.sort()
list_trigger_action

[['accelerationSensor', ['colorControl', 'colorControl.each']],
 ['accelerationSensor', ['colorControl', 'colorControl.setColor']],
 ['accelerationSensor', ['imageCapture', 'imageCapture.take']],
 ['accelerationSensor', ['momentary', 'momentary.push']],
 ['accelerationSensor', ['number', 'number.times']],
 ['accelerationSensor', ['sendAction', 'sendNotificationToContacts']],
 ['accelerationSensor', ['sendAction', 'sendNotificationToContacts']],
 ['accelerationSensor', ['sendAction', 'sendNotificationToContacts']],
 ['accelerationSensor', ['sendAction', 'sendNotificationToContacts']],
 ['accelerationSensor', ['sendAction', 'sendNotificationToContacts']],
 ['accelerationSensor', ['sendAction', 'sendNotificationToContacts']],
 ['accelerationSensor', ['sendAction', 'sendPush']],
 ['accelerationSensor', ['sendAction', 'sendPush']],
 ['accelerationSensor', ['sendAction', 'sendPush']],
 ['accelerationSensor', ['sendAction', 'sendSms']],
 ['accelerationSensor', ['sendAction', 'sendSms']],
 ['a

In [6]:
with open("../1_application_analysis/2_physical_channel_identification/set_of_capability_related_physical_channels.JSON","r") as inputfile:
    dict_channels_capabilities = json.load(inputfile)
dict_channels_capabilities

{'temperature': ['temperatureMeasurement',
  'thermostat',
  'location',
  'contactSensor',
  'motionSensor'],
 'humidity': ['presenceSensor',
  'waterSensor',
  'relativeHumidityMeasurement',
  'temperatureMeasurement',
  'contactSensor',
  'motionSensor'],
 'illumination': ['switch', 'accelerationSensor', 'location', 'contactSensor'],
 'location': ['beacon',
  'presenceSensor',
  'motionSensor',
  'contactSensor',
  'smokeDetector',
  'carbonMonoxideDetector'],
 'motion': ['motionSensor',
  'waterSensor',
  'contactSensor',
  'switch',
  'accelerationSensor',
  'presenceSensor'],
 'smoke': ['threeAxis'],
 'leakage': ['contactSensor', 'accelerationSensor', 'touchSensor']}

In [16]:
system_capabilities = [ "capability.app","time", "sendAction", "locationMode", "hub",
                        "capability.switch", "capability.lock", "capability.button",
                        "capability.powerMeter","device.jawboneUser", "capability.alarm"]

In [17]:
# find all the channel tuple 
channel_tuple = []
for trig_act in list_trigger_action: # take every pair trigger-action
    trigger_cap = trig_act[0]
    action_cap = trig_act[1][0]
    # find the channel where the capability came from
    if (trigger_cap in system_capabilities) and (action_cap in system_capabilities):
        channel_tuple.append([trigger_cap, action_cap])
        
    elif (not(trigger_cap in system_capabilities)) and (action_cap in system_capabilities):
        for tri_channel in dict_channels_capabilities.keys(): # check in every physical channel 
            if trigger_cap in dict_channels_capabilities[tri_channel]: # if there is the trigger capability
                channel_tuple.append([tri_channel, action_cap])
                
    elif trigger_cap in system_capabilities and (not(action_cap in system_capabilities)):
        for act_channel in dict_channels_capabilities.keys(): # check in every physical channel 
            if action_cap in dict_channels_capabilities[act_channel]: # if there is the action capability
                channel_tuple.append([trigger_cap, act_channel])
    else:
        for tri_channel in dict_channels_capabilities.keys(): # check in every physical channel 
            if trigger_cap in dict_channels_capabilities[tri_channel]: # if there is the trigger capability 
                for cap_channel in dict_channels_capabilities.keys(): # check in every physical channel 
                    if action_cap in dict_channels_capabilities[cap_channel]: # if there is the action capability
                        channel_tuple.append([tri_channel, cap_channel]) 
print(len(channel_tuple))
channel_tuple.sort()
channel_tuple

599


[['capability.alarm', 'capability.alarm'],
 ['capability.app', 'capability.alarm'],
 ['capability.app', 'capability.lock'],
 ['capability.app', 'capability.lock'],
 ['capability.app', 'capability.switch'],
 ['capability.app', 'capability.switch'],
 ['capability.app', 'capability.switch'],
 ['capability.app', 'capability.switch'],
 ['capability.app', 'capability.switch'],
 ['capability.app', 'temperature'],
 ['capability.app', 'temperature'],
 ['capability.app', 'temperature'],
 ['capability.app', 'temperature'],
 ['capability.button', 'locationMode'],
 ['capability.button', 'sendAction'],
 ['capability.button', 'sendAction'],
 ['capability.powerMeter', 'capability.switch'],
 ['capability.powerMeter', 'sendAction'],
 ['capability.powerMeter', 'sendAction'],
 ['capability.powerMeter', 'sendAction'],
 ['capability.switch', 'capability.switch'],
 ['capability.switch', 'capability.switch'],
 ['capability.switch', 'capability.switch'],
 ['capability.switch', 'capability.switch'],
 ['capabili

In [18]:
def pair2frequency(pair): # utility function for sort the set of piar ch, ch
    return pair[1]

result = []
tuple_found = []

for i in channel_tuple:
    if not i in tuple_found:
        frequency = channel_tuple.count(i) # count how much pair of (channel,channel) there are
        tuple_found.append(i)
        result.append([i, frequency])

result.sort(reverse=True, key=pair2frequency) # sort
result

[[['location', 'sendAction'], 58],
 [['leakage', 'sendAction'], 55],
 [['temperature', 'sendAction'], 53],
 [['temperature', 'capability.switch'], 43],
 [['location', 'capability.switch'], 40],
 [['illumination', 'sendAction'], 34],
 [['illumination', 'capability.switch'], 25],
 [['leakage', 'capability.switch'], 24],
 [['motion', 'sendAction'], 22],
 [['humidity', 'capability.switch'], 21],
 [['humidity', 'sendAction'], 20],
 [['time', 'sendAction'], 18],
 [['temperature', 'temperature'], 16],
 [['motion', 'capability.switch'], 15],
 [['location', 'temperature'], 12],
 [['humidity', 'temperature'], 10],
 [['illumination', 'temperature'], 10],
 [['motion', 'temperature'], 7],
 [['capability.switch', 'sendAction'], 6],
 [['location', 'capability.alarm'], 6],
 [['location', 'capability.lock'], 6],
 [['location', 'locationMode'], 6],
 [['smoke', 'capability.switch'], 6],
 [['smoke', 'sendAction'], 6],
 [['temperature', 'capability.alarm'], 6],
 [['capability.app', 'capability.switch'], 5]

In [19]:
# value assignment to physical channel
# from cap 4.4.1 - p837
# The procedure of the value assignment starts from the channel with the highest frequency in all tuples, 
# where we assign an initial value K (K can be an arbitrary value) to the first channel.
# Then, we assign a value to the next channel, which has the highest co-occurrence with the first 
# assigned channel in all channel tuples. We define a step length, denoted by λ.

channel_analysed = []
channel_values = []
k = 15
delta = 15

for i in range(len(result)):
    channel = result[i][0][0] # fist channel of the pair
    if not channel in system_capabilities: # analyse only physical channels
        if not channel in channel_analysed:
            channel_analysed.append(channel)
            channel_values.append(k)
            k += delta
    channel = result[i][0][1] # second channel of the pair
    if not channel in system_capabilities: # analyse only physical channels
        if not channel in channel_analysed:
            channel_analysed.append(channel)
            channel_values.append(k)
            k += delta
        
print(channel_analysed)
print(channel_values)

['location', 'leakage', 'temperature', 'illumination', 'motion', 'humidity', 'smoke']
[15, 30, 45, 60, 75, 90, 105]


In [20]:
# value assignment to system channel
# from cap 4.4.1 - p837 "In our design, for value assignment to a system channel, we consider all its
# related physical channels where its value is assigned to be the sum of all associated physical channels’ values."
system_channel = []
for app in trigger_action.keys(): # for every pair of trigger-action I've got
    triggers = trigger_action[app].keys()
    for cap_trigger in triggers:
        if cap_trigger in system_capabilities: # if the cap is about system channels
            system_value = 0
            for action in trigger_action[app][cap_trigger]:
                if action[0] in channel_analysed:
                    system_value += channel_values[channel_analysed.index(action[0])] # sum the physical channel correlated value
            system_channel.append([cap_trigger, system_value])
                    
system_channel

[['capability.app', 0],
 ['capability.app', 0],
 ['capability.button', 0],
 ['time', 0],
 ['capability.switch', 0],
 ['time', 0],
 ['capability.powerMeter', 0],
 ['capability.powerMeter', 0],
 ['time', 0],
 ['capability.app', 0],
 ['capability.switch', 0],
 ['capability.switch', 0],
 ['time', 0],
 ['time', 0],
 ['hub', 0],
 ['capability.app', 0],
 ['time', 0],
 ['capability.app', 0],
 ['time', 0],
 ['capability.button', 0],
 ['capability.app', 0],
 ['time', 0],
 ['capability.switch', 0],
 ['capability.switch', 0],
 ['time', 0],
 ['device.jawboneUser', 0],
 ['time', 0],
 ['capability.alarm', 0],
 ['capability.app', 0],
 ['capability.switch', 0],
 ['capability.switch', 0],
 ['time', 0]]

In [23]:
# vector value assignment
# To distinguish them in a behavior vector, as long as a channel is
# associated with any trigger capability, we multiply a co-efficient
# “−1” with its channel value.
set_of_vectors = []
pair_vector = [0, 0, 0, 0, 0, 0, 0] # I use a 7-vector becouse I don't have any system interaction ...

for pair in channel_tuple: # for every tuple (ch, ch)
    if (pair[0] in channel_analysed) and (pair[1] in channel_analysed): # channel analysed should have physical + system channels ....
        index_trigger_channel = channel_analysed.index(pair[0])
        trigger_value = -1 * channel_values[index_trigger_channel]
        pair_vector[index_trigger_channel] = trigger_value
        
        index_action_channel = channel_analysed.index(pair[1])
        action_value = channel_values[index_action_channel]
        pair_vector[index_action_channel] = action_value
        
        if not pair_vector in set_of_vectors: # avoid duplicates 
            set_of_vectors.append(pair_vector)
        pair_vector = [0, 0, 0, 0, 0, 0, 0]

set_of_vectors

[[0, 0, 45, 0, 0, -90, 0],
 [0, 0, 45, -60, 0, 0, 0],
 [0, -30, 45, 0, 0, 0, 0],
 [-15, 0, 45, 0, 0, 0, 0],
 [0, 0, 45, 0, -75, 0, 0],
 [0, 0, 45, 0, 0, 0, -105],
 [0, 0, 45, 0, 0, 0, 0]]

In [24]:
# similarity calculation
from sklearn.cluster import KMeans

np_set_of_vectors = np.array(set_of_vectors) # in this case i've got only 7 vectors ....
cluster = np_set_of_vectors.shape[1]
max_iter = 500

sk_kmeans = KMeans(n_clusters=cluster, max_iter=max_iter).fit(np_set_of_vectors)
sk_labels = sk_kmeans.labels_
sk_centroids = sk_kmeans.cluster_centers_
print(sk_labels)

[3 4 5 6 1 2 0]


In [ ]:
# risk evaluation --> risk score
# the paper doesn't describe how calculate the T vector
# For inter-app interaction chains, we combine the values of intra-app interaction vectors.
# For the bridging channels between intra-app interactions, we treat them as a combination
# of trigger conditions, all of which multiply a negative coefficient in vector value
# assignment.